In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import joblib
import warnings
warnings.filterwarnings('ignore')

# Importar transformadores personalizados
import os, sys
sys.path.append(r"C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\notebooks")

from custom_transformers import (
    TemporalFeatures, LagCreator, OutlierTreatment, 
    LogTransformation, CategoricalEncoder, ToNumericTransformer
)

print("="*80)
print("CREACIÓN DEL PIPELINE DE INGENIERÍA DE CARACTERÍSTICAS")
print("="*80)

# ==================================================================================
# DEFINICIÓN DE VARIABLES Y CONFIGURACIÓN
# ==================================================================================

OUTLIER_VARS = ['Unidades_Vendidas']
LOG_TRANSFORM_VARS = ['Unidades_Vendidas_tratado']

# ==================================================================================
# CONSTRUCCIÓN DEL PIPELINE
# ==================================================================================

ventas_preprocessing_pipeline = Pipeline([
    ('temporal_features', TemporalFeatures()),
    ('lag_creator', LagCreator(lags=[1, 7, 14, 30])),
    ('categorical_encoder', CategoricalEncoder()),
    ('outlier_treatment', OutlierTreatment(columns=OUTLIER_VARS)),
    ('log_transformation', LogTransformation(columns=LOG_TRANSFORM_VARS)),
    ('to_numeric', ToNumericTransformer()),
    ('numerical_imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

print("\nPipeline de preprocesamiento creado exitosamente")
print(f"Pasos del pipeline: {len(ventas_preprocessing_pipeline.steps)}")

# ==================================================================================
# CARGA Y PREPARACIÓN DE DATOS PARA AJUSTE
# ==================================================================================

print("\n" + "="*80)
print("CARGANDO DATOS PARA AJUSTAR EL PIPELINE")
print("="*80)

df = pd.read_csv(r'C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\data\raw\ventas.csv')
print(f"Datos cargados: {len(df):,} registros")
print(f"Columnas: {list(df.columns)}")

# Convertir fecha antes de pasar al pipeline
df['Fecha_Venta'] = pd.to_datetime(df['Fecha_Venta'], dayfirst=True, errors='coerce', format='mixed')
df = df.sort_values(['Codigo_Sucursal', 'Codigo_Producto', 'Fecha_Venta'])

# Ajustar el pipeline con los datos
X_sample = df.drop(['Total'], axis=1) if 'Total' in df.columns else df
ventas_preprocessing_pipeline.fit(X_sample)

print("\nPipeline ajustado exitosamente")

# ==================================================================================
# VERIFICAR QUE EL PIPELINE FUNCIONA
# ==================================================================================

print("\n" + "="*80)
print("VERIFICACIÓN DEL PIPELINE")
print("="*80)

# Transformar una muestra pequeña para verificar
X_transformed = ventas_preprocessing_pipeline.transform(X_sample.head(100))
print(f"Shape después de transformación: {X_transformed.shape}")
print(f"Tipo de dato: {type(X_transformed)}")
print(f"Es array numpy: {isinstance(X_transformed, np.ndarray)}")

# ==================================================================================
# EXPORTAR PIPELINE
# ==================================================================================

pipeline_path = r'C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\models\ventas_preprocessing_pipeline.pkl'
joblib.dump(ventas_preprocessing_pipeline, pipeline_path)

print("\n" + "="*80)
print("PIPELINE GUARDADO")
print("="*80)
print(f"Ubicación: {pipeline_path}")
print("\nEl pipeline está listo para ser usado en entrenamiento e inferencia")

print("\n" + "="*80)
print("RESUMEN DEL PIPELINE")
print("="*80)
print("""
PASOS DEL PIPELINE:
1. Creación de características temporales (elimina Fecha_Venta)
2. Creación de lags (1, 7, 14, 30 días) y rolling means
3. Codificación de variables categóricas (elimina originales)
4. Tratamiento de outliers (elimina Unidades_Vendidas original)
5. Transformación logarítmica
6. Conversión a array numérico
7. Imputación de valores faltantes (mediana)
8. Escalado de características (StandardScaler)

OUTPUT: Array numpy con todas las características numéricas procesadas
""")

CREACIÓN DEL PIPELINE DE INGENIERÍA DE CARACTERÍSTICAS

Pipeline de preprocesamiento creado exitosamente
Pasos del pipeline: 8

CARGANDO DATOS PARA AJUSTAR EL PIPELINE
Datos cargados: 73,153 registros
Columnas: ['Codigo_Sucursal', 'Fecha_Venta', 'Codigo_Producto', 'Unidades_Vendidas', 'Total']

Pipeline ajustado exitosamente

VERIFICACIÓN DEL PIPELINE
Shape después de transformación: (100, 16)
Tipo de dato: <class 'numpy.ndarray'>
Es array numpy: True

PIPELINE GUARDADO
Ubicación: C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\models\ventas_preprocessing_pipeline.pkl

El pipeline está listo para ser usado en entrenamiento e inferencia

RESUMEN DEL PIPELINE

PASOS DEL PIPELINE:
1. Creación de características temporales (elimina Fecha_Venta)
2. Creación de lags (1, 7, 14, 30 días) y rolling means
3. Codificación de variables categóricas (elimina originales)
4. Tratamiento de outliers (elimina Unidades_Vendidas original)
5. Transfor

In [3]:
import importlib, sys
sys.path.append(r"C:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\notebooks")

import custom_transformers
importlib.reload(custom_transformers)

from custom_transformers import (
    TemporalFeatures, LagCreator, OutlierTreatment, 
    LogTransformation, CategoricalEncoder, DropColumnsTransformer
)



ImportError: cannot import name 'DropColumnsTransformer' from 'custom_transformers' (c:\Users\hsuna\Desktop\Proyecto Final Product Development\proyecto final v2\repo-seriestemporales-g4-pd\notebooks\custom_transformers.py)